# Lab 7: Agent Memory & Planning

**Duration**: 100-130 minutes  
**Level**: Advanced  
**Prerequisites**: Lab 6 completed

## Overview

In this lab, you'll build increasingly sophisticated agents with memory systems and planning capabilities.

### What You'll Build:

1. **Memory Agent** - Agent with short-term, working, and long-term memory
2. **ReAct Agent** - Agent using Thought → Action → Observation loop
3. **Planning Agent** - Agent that plans before executing
4. **Reflective Agent** - Agent with self-reflection and error correction
5. **IntelliAgent v1.0** - Complete agent with memory + planning (Capstone)

## Learning Objectives

- Implement short-term memory (conversation history)
- Build working memory for task tracking
- Create long-term memory with vector databases
- Implement the ReAct framework
- Build planning agents that create strategies
- Add self-reflection for error correction
- Combine memory and planning in production agents

## Setup

In [ ]:
# Install required packages
!pip install openai anthropic python-dotenv chromadb sentence-transformers

In [ ]:
# Import required libraries
import os
import json
import logging
from datetime import datetime
from typing import List, Dict, Any, Optional, Callable
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API keys
openai_key = os.getenv("OPENAI_API_KEY")

print(f"OpenAI API Key present: {bool(openai_key)}")

In [ ]:
# Test setup
from openai import OpenAI
import chromadb

# Test OpenAI connection
try:
    client = OpenAI(api_key=openai_key)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Hi"}],
        max_tokens=10
    )
    print("✓ OpenAI API working")
except Exception as e:
    print(f"✗ OpenAI API error: {e}")

# Test ChromaDB
try:
    chroma_client = chromadb.Client()
    print("✓ ChromaDB working")
except Exception as e:
    print(f"✗ ChromaDB error: {e}")

print("\n✅ Setup complete! Ready for exercises.")

## Exercise 1: Agent with Memory Systems

Build an agent with short-term, working, and long-term memory.

### Part A: Short-Term Memory

Short-term memory manages conversation history.

In [ ]:
# Short-Term Memory Implementation

class ShortTermMemory:
    """Manages conversation history"""

    def __init__(self, max_messages: int = 20):
        self.messages: List[Dict] = []
        self.max_messages = max_messages

    def add_system_message(self, content: str):
        """Add system message at the beginning"""
        self.messages.insert(0, {"role": "system", "content": content})

    def add_user_message(self, content: str):
        """Add user message"""
        self.messages.append({"role": "user", "content": content})
        self._trim_if_needed()

    def add_assistant_message(self, content: str):
        """Add assistant message"""
        self.messages.append({"role": "assistant", "content": content})
        self._trim_if_needed()

    def _trim_if_needed(self):
        """Keep only recent messages + system message"""
        system_msgs = [m for m in self.messages if m["role"] == "system"]
        other_msgs = [m for m in self.messages if m["role"] != "system"]

        if len(other_msgs) > self.max_messages:
            other_msgs = other_msgs[-self.max_messages:]

        self.messages = system_msgs + other_msgs

    def get_messages(self) -> List[Dict]:
        """Get all messages"""
        return self.messages

    def clear(self):
        """Clear all except system messages"""
        system_msgs = [m for m in self.messages if m["role"] == "system"]
        self.messages = system_msgs

print("ShortTermMemory class defined!")

In [ ]:
# Conversational Agent with Short-Term Memory

class ConversationalAgent:
    """Agent with conversation memory"""

    def __init__(self, system_prompt: str = "You are a helpful assistant."):
        self.memory = ShortTermMemory(max_messages=20)
        self.memory.add_system_message(system_prompt)

    def chat(self, user_message: str) -> str:
        """Chat with memory"""
        print(f"\n{'='*60}")
        print(f"User: {user_message}")

        # Add to memory
        self.memory.add_user_message(user_message)

        # Get response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.memory.get_messages()
        )

        assistant_message = response.choices[0].message.content
        self.memory.add_assistant_message(assistant_message)

        print(f"Agent: {assistant_message}")
        print('='*60)

        return assistant_message

    def reset(self):
        """Reset conversation"""
        self.memory.clear()
        print("🔄 Conversation reset")

print("ConversationalAgent class defined!")

In [ ]:
# Test Short-Term Memory

agent = ConversationalAgent()

print("\n" + "="*60)
print("CONVERSATIONAL AGENT WITH SHORT-TERM MEMORY")
print("="*60)

# Conversation demonstrating memory
agent.chat("Hi, my name is Alice")
agent.chat("What's my name?")  # Should remember: Alice
agent.chat("I love Python programming")
agent.chat("What do I love?")  # Should remember: Python

print("\n" + "="*60)
print("✅ Exercise 1A Complete!")
print("="*60)

### Part B: Working Memory

Working memory manages task context and progress.

In [ ]:
# Working Memory Implementation

class WorkingMemory:
    """Manages task context and progress"""

    def __init__(self):
        self.task_name: Optional[str] = None
        self.task_status: str = "idle"
        self.variables: Dict[str, Any] = {}
        self.steps_completed: list = []
        self.current_step: Optional[str] = None
        self.started_at: Optional[datetime] = None
        self.completed_at: Optional[datetime] = None

    def start_task(self, task_name: str):
        """Start a new task"""
        self.task_name = task_name
        self.task_status = "in_progress"
        self.started_at = datetime.now()
        self.variables = {}
        self.steps_completed = []
        print(f"\n📋 Started task: {task_name}")

    def set_variable(self, key: str, value: Any):
        """Store a variable"""
        self.variables[key] = value
        print(f"   💾 Stored: {key} = {value}")

    def get_variable(self, key: str) -> Any:
        """Retrieve a variable"""
        return self.variables.get(key)

    def complete_step(self, step_name: str):
        """Mark a step as completed"""
        self.steps_completed.append({
            "step": step_name,
            "completed_at": datetime.now()
        })
        self.current_step = None
        print(f"   ✅ Completed step: {step_name}")

    def start_step(self, step_name: str):
        """Start a new step"""
        self.current_step = step_name
        print(f"   🔄 Starting step: {step_name}")

    def complete_task(self, success: bool = True):
        """Complete the task"""
        self.task_status = "completed" if success else "failed"
        self.completed_at = datetime.now()

        duration = (self.completed_at - self.started_at).total_seconds()
        print(f"\n{'✅' if success else '❌'} Task {self.task_status}: {self.task_name}")
        print(f"   Duration: {duration:.2f}s")
        print(f"   Steps completed: {len(self.steps_completed)}")

    def get_summary(self) -> Dict[str, Any]:
        """Get task summary"""
        return {
            "task_name": self.task_name,
            "status": self.task_status,
            "steps_completed": len(self.steps_completed),
            "variables": self.variables
        }

print("WorkingMemory class defined!")

In [ ]:
# Test Working Memory

memory = WorkingMemory()

print("\n" + "="*60)
print("WORKING MEMORY DEMONSTRATION")
print("="*60)

# Simulate a multi-step task
memory.start_task("Calculate compound interest")

memory.start_step("Get input values")
memory.set_variable("principal", 1000)
memory.set_variable("rate", 0.05)
memory.set_variable("time", 3)
memory.complete_step("Get input values")

memory.start_step("Calculate final amount")
principal = memory.get_variable("principal")
rate = memory.get_variable("rate")
time = memory.get_variable("time")
amount = principal * ((1 + rate) ** time)
memory.set_variable("final_amount", amount)
memory.complete_step("Calculate final amount")

memory.start_step("Calculate interest gained")
interest = amount - principal
memory.set_variable("interest", interest)
memory.complete_step("Calculate interest gained")

memory.complete_task(success=True)

print("\n" + "="*60)
print("Task Summary:")
print(json.dumps(memory.get_summary(), indent=2))
print("="*60)

print("\n✅ Exercise 1B Complete!")

### Part C: Long-Term Memory with ChromaDB

Long-term memory uses a vector database for persistent storage with semantic search.

In [ ]:
# Long-Term Memory Implementation

import chromadb
from sentence_transformers import SentenceTransformer

class LongTermMemory:
    """Manages persistent memory with vector database"""

    def __init__(self, collection_name: str = "agent_memory"):
        # Initialize ChromaDB
        self.client = chromadb.PersistentClient(path="./agent_memory_db")

        # Initialize embedding model
        self.embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            metadata={"description": "Agent long-term memory"}
        )

        print(f"✓ Long-term memory initialized")
        print(f"  Collection: {collection_name}")
        print(f"  Memories stored: {self.collection.count()}")

    def store_memory(
        self,
        content: str,
        memory_type: str = "general",
        metadata: Dict = None
    ) -> str:
        """Store a memory"""
        # Generate embedding
        embedding = self.embedding_model.encode(content)

        # Create unique ID
        memory_id = f"mem_{datetime.now().timestamp()}"

        # Prepare metadata
        mem_metadata = {
            "type": memory_type,
            "created_at": datetime.now().isoformat(),
            **(metadata or {})
        }

        # Store in ChromaDB
        self.collection.add(
            documents=[content],
            embeddings=[embedding.tolist()],
            ids=[memory_id],
            metadatas=[mem_metadata]
        )

        print(f"💾 Stored: {content[:50]}{'...' if len(content) > 50 else ''}")
        return memory_id

    def retrieve_memories(
        self,
        query: str,
        n_results: int = 5,
        memory_type: str = None
    ) -> List[Dict]:
        """Retrieve relevant memories"""
        # Generate query embedding
        query_embedding = self.embedding_model.encode(query)

        # Build filter
        where_filter = {"type": memory_type} if memory_type else None

        # Search
        results = self.collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=n_results,
            where=where_filter,
            include=["documents", "metadatas", "distances"]
        )

        # Format results
        memories = []
        if results['documents'] and results['documents'][0]:
            for i in range(len(results['documents'][0])):
                memories.append({
                    "content": results['documents'][0][i],
                    "metadata": results['metadatas'][0][i],
                    "relevance": 1 / (1 + results['distances'][0][i])
                })

        return memories

    def clear_memories(self):
        """Clear all memories"""
        self.client.delete_collection(self.collection.name)
        self.collection = self.client.create_collection(
            name=self.collection.name,
            metadata={"description": "Agent long-term memory"}
        )
        print("🗑️  Cleared all memories")

print("LongTermMemory class defined!")

In [ ]:
# Test Long-Term Memory

ltm = LongTermMemory()

print("\n" + "="*60)
print("LONG-TERM MEMORY DEMONSTRATION")
print("="*60)

# Store some memories
print("\n📝 Storing memories...")
ltm.store_memory(
    "User's name is Alice",
    memory_type="user_fact"
)

ltm.store_memory(
    "User likes Python programming",
    memory_type="user_preference"
)

ltm.store_memory(
    "User works as a data scientist",
    memory_type="user_fact"
)

ltm.store_memory(
    "User prefers detailed explanations",
    memory_type="user_preference"
)

# Retrieve relevant memories
print("\n🔍 Querying memories...")
print("\nQuery: 'What does the user do?'")
memories = ltm.retrieve_memories("What does the user do?", n_results=3)

for i, mem in enumerate(memories):
    print(f"\n[{i+1}] Relevance: {mem['relevance']:.4f}")
    print(f"    Content: {mem['content']}")
    print(f"    Type: {mem['metadata']['type']}")

print("\n" + "="*60)
print("✅ Exercise 1C Complete!")
print("="*60)

### 🎯 Checkpoint 1

**What you learned:**
- How to implement short-term memory for conversation history
- How to use working memory for task tracking
- How to build long-term memory with vector databases
- Memory trimming and management strategies

**Key Differences:**
- **Short-term memory**: Recent conversation history (trimmed)
- **Working memory**: Current task state and variables
- **Long-term memory**: Persistent facts with semantic search

## Exercise 2: ReAct Agent

Build an agent using the **Thought → Action → Observation** loop.

ReAct (Reasoning + Acting) enables transparent, step-by-step problem solving.

In [ ]:
# Tool Implementations for ReAct Agent

import ast
import operator

def safe_eval_math(expression: str) -> float:
    """Safely evaluate mathematical expressions without using eval()"""
    # Define allowed operators
    operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
    }
    
    def eval_node(node):
        if isinstance(node, ast.Num):  # number
            return node.n
        elif isinstance(node, ast.BinOp):  # binary operation
            return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
        elif isinstance(node, ast.UnaryOp):  # unary operation
            return operators[type(node.op)](eval_node(node.operand))
        else:
            raise ValueError(f"Unsupported operation: {type(node)}")
    
    try:
        tree = ast.parse(expression, mode='eval')
        return eval_node(tree.body)
    except Exception as e:
        raise ValueError(f"Invalid mathematical expression: {str(e)}")

def calculator(expression: str) -> Dict[str, Any]:
    """Calculator tool with safe evaluation"""
    try:
        result = safe_eval_math(expression)
        return {"success": True, "result": result}
    except Exception as e:
        return {"success": False, "error": str(e)}

def search_info(query: str) -> Dict[str, Any]:
    """Simulated search tool"""
    knowledge = {
        "paris": "Paris is the capital of France with population of ~2.2 million (city proper). Founded in 3rd century BC.",
        "python": "Python is a high-level programming language created by Guido van Rossum in 1991. Known for readability.",
        "ai": "AI (Artificial Intelligence) is the simulation of human intelligence by machines and computer systems.",
        "machine learning": "Machine learning is a subset of AI that enables systems to learn from data without explicit programming."
    }

    query_lower = query.lower()
    for key, value in knowledge.items():
        if key in query_lower:
            return {"success": True, "result": value}

    return {"success": False, "result": "No information found"}

print("Tools defined!")

In [ ]:
# Tool Definitions for OpenAI

REACT_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Perform mathematical calculations",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to evaluate"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_info",
            "description": "Search for information on a topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

REACT_FUNCTIONS = {
    "calculator": calculator,
    "search_info": search_info
}

print("Tool definitions ready!")

In [ ]:
# ReAct Agent Implementation

def react_agent(user_query: str, verbose: bool = True):
    """
    ReAct agent showing explicit reasoning
    """
    if verbose:
        print(f"\n{'='*70}")
        print(f"USER QUERY: {user_query}")
        print('='*70)

    # System prompt for ReAct
    system_prompt = """You are a helpful assistant using the ReAct (Reasoning + Acting) framework.

For each step:
1. THINK about what to do next
2. Use a tool if needed (ACTION)
3. OBSERVE the result
4. DECIDE if you have enough information

Show your reasoning clearly by thinking step-by-step before each action."""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]

    iteration = 0
    max_iterations = 5

    while iteration < max_iterations:
        iteration += 1

        if verbose:
            print(f"\n{'─'*70}")
            print(f"ITERATION {iteration}")
            print('─'*70)

        # Get response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=REACT_TOOLS,
            tool_choice="auto"
        )

        response_message = response.choices[0].message

        # Check if done
        if not response_message.tool_calls:
            final_answer = response_message.content

            if verbose:
                print(f"\n💭 FINAL THOUGHT:")
                print(final_answer)
                print('='*70)

            return final_answer

        # Show thinking
        if verbose and response_message.content:
            print(f"\n💭 THOUGHT:")
            print(response_message.content)

        # Process tool calls (ACTIONS)
        messages.append(response_message)

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            if verbose:
                print(f"\n🔧 ACTION: {function_name}")
                print(f"   Args: {json.dumps(arguments)}")

            # Execute tool
            function = REACT_FUNCTIONS[function_name]
            result = function(**arguments)

            if verbose:
                print(f"\n👁️  OBSERVATION:")
                print(f"   {json.dumps(result, indent=3)}")

            # Add result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": json.dumps(result)
            })

    return "Max iterations reached"

print("ReAct agent defined!")

In [ ]:
# Test ReAct Agent

print("\n" + "="*70)
print("REACT AGENT DEMONSTRATION")
print("="*70)

# Test 1: Simple calculation
print("\n" + "#"*70)
print("TEST 1: Simple Calculation")
print("#"*70)
react_agent("What is 25% of 840?")

# Test 2: Information retrieval
print("\n" + "#"*70)
print("TEST 2: Information Retrieval")
print("#"*70)
react_agent("Tell me about Python programming language")

# Test 3: Multi-step reasoning
print("\n" + "#"*70)
print("TEST 3: Multi-Step Reasoning")
print("#"*70)
react_agent("Search for Paris population, then calculate how many people per 100,000")

print("\n" + "="*70)
print("✅ Exercise 2 Complete!")
print("="*70)

### 🎯 Checkpoint 2

**What you learned:**
- The ReAct (Reasoning + Acting) framework
- Thought → Action → Observation loop
- Explicit reasoning for transparency
- Multi-step problem solving

**Key Insight**: ReAct makes agent reasoning visible, making it easier to debug and understand decision-making.

## Exercise 3: Planning Agent

Build an agent that **creates a plan before executing**.

This is the **plan-then-execute** pattern.

In [ ]:
# Planning Agent Implementation

class PlanningAgent:
    """Agent that plans before executing"""

    def __init__(self, tools, functions):
        self.tools = tools
        self.functions = functions
        self.plan = []
        self.execution_log = []

    def create_plan(self, user_query: str) -> List[str]:
        """Create a step-by-step plan"""
        print(f"\n{'='*70}")
        print("PLANNING PHASE")
        print('='*70)

        planning_prompt = f"""Given this task, create a detailed step-by-step plan.

Task: {user_query}

Available tools:
- calculator: for mathematical calculations
- search_info: for finding information

Create a numbered plan with specific steps. Each step should be clear and actionable."""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a planning assistant. Create clear, actionable plans."},
                {"role": "user", "content": planning_prompt}
            ]
        )

        plan_text = response.choices[0].message.content
        print(f"\n📋 PLAN:")
        print(plan_text)

        # Parse plan into steps
        lines = plan_text.split('\n')
        steps = [line.strip() for line in lines if line.strip() and any(c.isdigit() for c in line[:3])]

        self.plan = steps
        return steps

    def execute_plan(self) -> Dict:
        """Execute the created plan"""
        print(f"\n{'='*70}")
        print("EXECUTION PHASE")
        print('='*70)

        messages = [
            {"role": "system", "content": "Execute the plan step by step. Follow each step carefully."},
            {"role": "user", "content": f"Execute this plan:\n" + "\n".join(self.plan)}
        ]

        step_num = 0
        max_iterations = len(self.plan) + 5

        while step_num < max_iterations:
            step_num += 1
            print(f"\n{'─'*70}")
            print(f"EXECUTION STEP {step_num}")
            print('─'*70)

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                tools=self.tools,
                tool_choice="auto"
            )

            response_message = response.choices[0].message

            if response_message.content:
                print(f"💭 {response_message.content[:150]}...")

            # Check if done
            if not response_message.tool_calls:
                final_answer = response_message.content
                print(f"\n✅ Execution complete!")
                print(f"\nFinal Answer:\n{final_answer}")
                return {
                    "success": True,
                    "answer": final_answer,
                    "plan": self.plan,
                    "execution_log": self.execution_log
                }

            # Execute tools
            messages.append(response_message)

            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)

                print(f"\n🔧 Tool: {function_name}")
                print(f"   Args: {json.dumps(arguments)}")

                result = self.functions[function_name](**arguments)

                print(f"   Result: {json.dumps(result)}")

                self.execution_log.append({
                    "step": step_num,
                    "tool": function_name,
                    "arguments": arguments,
                    "result": result
                })

                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(result)
                })

        return {
            "success": False,
            "error": "Max iterations reached"
        }

    def plan_and_execute(self, user_query: str) -> Dict:
        """Complete plan-then-execute workflow"""
        # Phase 1: Planning
        self.create_plan(user_query)

        # Phase 2: Execution
        result = self.execute_plan()

        return result

print("PlanningAgent class defined!")

In [ ]:
# Test Planning Agent

print("\n" + "="*70)
print("PLANNING AGENT DEMONSTRATION")
print("="*70)

agent = PlanningAgent(REACT_TOOLS, REACT_FUNCTIONS)

result = agent.plan_and_execute(
    "Find information about machine learning, then calculate how many years it's been since AI was coined in 1956 (current year 2024)"
)

print(f"\n{'='*70}")
print("SUMMARY")
print('='*70)
print(f"Success: {result['success']}")
print(f"Steps in plan: {len(result.get('plan', []))}")
print(f"Tools executed: {len(result.get('execution_log', []))}")

print("\n✅ Exercise 3 Complete!")

### 🎯 Checkpoint 3

**What you learned:**
- Plan-then-execute pattern
- Creating structured plans from goals
- Executing plans systematically
- Tracking execution progress

**When to use planning**: Complex multi-step tasks benefit from upfront planning to avoid getting stuck or taking inefficient paths.

## Exercise 4: Self-Reflective Agent

Build an agent that **reflects on its progress** and adjusts course.

In [ ]:
# Reflective Agent Implementation

class ReflectiveAgent:
    """Agent with self-reflection capabilities"""

    def __init__(self, tools, functions):
        self.tools = tools
        self.functions = functions
        self.reflection_history = []
        self.action_history = []

    def reflect(self, action_history: List[Dict]) -> str:
        """Reflect on actions taken"""
        if not action_history:
            return ""

        history_text = "Actions taken so far:\n"
        for i, action in enumerate(action_history):
            history_text += f"{i+1}. {action['tool']}({json.dumps(action['args'])}) → {action['result']}\n"

        reflection_prompt = f"""{history_text}

Reflect on these actions:
1. Are we making progress toward the goal?
2. Did any action fail or produce unexpected results?
3. Should we change our approach?
4. What should we do next?

Provide a brief reflection."""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a reflective assistant that evaluates progress."},
                {"role": "user", "content": reflection_prompt}
            ]
        )

        reflection = response.choices[0].message.content
        return reflection

    def run(self, user_query: str, reflection_interval: int = 3):
        """Run agent with periodic reflection"""
        print(f"\n{'='*70}")
        print(f"TASK: {user_query}")
        print('='*70)

        system_prompt = """You are a thoughtful ReAct agent that learns from mistakes.

Use the ReAct framework:
- THOUGHT: Reason about what to do
- ACTION: Use a tool
- OBSERVATION: Analyze the result
- REFLECTION: Evaluate if approach is working

If something fails, try a different approach."""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ]

        iteration = 0
        max_iterations = 10

        while iteration < max_iterations:
            iteration += 1
            print(f"\n{'─'*70}")
            print(f"ITERATION {iteration}")
            print('─'*70)

            # Periodic reflection
            if len(self.action_history) > 0 and len(self.action_history) % reflection_interval == 0:
                reflection = self.reflect(self.action_history)
                print(f"\n🤔 REFLECTION:")
                print(reflection)

                self.reflection_history.append({
                    "iteration": iteration,
                    "reflection": reflection
                })

                # Add reflection to context
                messages.append({
                    "role": "user",
                    "content": f"Reflection on progress: {reflection}"
                })

            # Get next action
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                tools=self.tools,
                tool_choice="auto"
            )

            response_message = response.choices[0].message

            if response_message.content:
                print(f"\n💭 {response_message.content}")

            # Check if done
            if not response_message.tool_calls:
                print(f"\n✅ COMPLETE")
                return {
                    "answer": response_message.content,
                    "action_history": self.action_history,
                    "reflections": self.reflection_history,
                    "iterations": iteration
                }

            # Execute actions
            messages.append(response_message)

            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)

                print(f"\n🔧 ACTION: {function_name}({json.dumps(arguments)})")

                # Execute
                try:
                    result = self.functions[function_name](**arguments)
                    success = result.get("success", True) if isinstance(result, dict) else True
                except Exception as e:
                    result = {"success": False, "error": str(e)}
                    success = False

                print(f"👁️  OBSERVATION: {json.dumps(result)}")

                if not success:
                    print("⚠️  Action failed!")

                # Record action
                self.action_history.append({
                    "tool": function_name,
                    "args": arguments,
                    "result": result,
                    "success": success
                })

                # Add to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(result)
                })

        return {
            "answer": "Max iterations reached",
            "action_history": self.action_history,
            "reflections": self.reflection_history
        }

print("ReflectiveAgent class defined!")

In [ ]:
# Test Reflective Agent

print("\n" + "="*70)
print("REFLECTIVE AGENT DEMONSTRATION")
print("="*70)

agent = ReflectiveAgent(REACT_TOOLS, REACT_FUNCTIONS)

result = agent.run(
    "Calculate 20% of 500, search for info about AI, then calculate 10% of the first result",
    reflection_interval=2
)

print(f"\n{'='*70}")
print("SUMMARY")
print('='*70)
print(f"Iterations: {result['iterations']}")
print(f"Actions taken: {len(result['action_history'])}")
print(f"Reflections made: {len(result['reflections'])}")

print("\n✅ Exercise 4 Complete!")

### 🎯 Checkpoint 4

**What you learned:**
- Self-reflection for progress evaluation
- Error detection and course correction
- Adaptive agent behavior
- Learning from mistakes

**Key Benefit**: Reflection allows agents to detect when they're stuck or going down the wrong path, enabling adaptive behavior.

## Capstone Project: IntelliAgent v1.0

Build a **complete intelligent agent** combining all concepts:
- Full memory system (short-term + working + long-term)
- ReAct framework with explicit reasoning
- Self-reflection and error correction
- Production-ready error handling

This is your **production-ready intelligent agent**!

In [ ]:
# IntelliAgent v1.0 - Complete Intelligent Agent

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class IntelliAgent:
    """
    Production-ready intelligent agent with:
    - Full memory system (short-term, working, long-term)
    - ReAct framework
    - Planning capabilities
    - Self-reflection
    """

    def __init__(self, name: str = "IntelliAgent"):
        self.name = name

        # Memory systems
        self.short_term = ShortTermMemory(max_messages=20)
        self.working = WorkingMemory()
        self.long_term = LongTermMemory(collection_name=f"{name.lower()}_memory")

        # Tools
        self.tools = self._setup_tools()
        self.functions = {
            "calculator": calculator,
            "search_info": search_info
        }

        # State
        self.action_history = []
        self.reflection_history = []

        # System prompt
        system_prompt = f"""You are {name}, an intelligent assistant with memory and planning capabilities.

You use the ReAct framework:
1. THOUGHT: Reason about what to do next
2. ACTION: Use tools when needed
3. OBSERVATION: Analyze results
4. REFLECTION: Periodically evaluate progress

You have access to:
- Short-term memory (conversation history)
- Working memory (task variables)
- Long-term memory (persistent facts)

Plan complex tasks before executing. Reflect on your progress. Learn from mistakes."""

        self.short_term.add_system_message(system_prompt)

        logger.info(f"{name} initialized with full capabilities")

    def _setup_tools(self):
        """Setup tool definitions"""
        return [
            {
                "type": "function",
                "function": {
                    "name": "calculator",
                    "description": "Perform mathematical calculations",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "expression": {"type": "string", "description": "Math expression"}
                        },
                        "required": ["expression"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_info",
                    "description": "Search for information",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "Search query"}
                        },
                        "required": ["query"]
                    }
                }
            }
        ]

    def remember(self, fact: str, memory_type: str = "user_fact"):
        """Store in long-term memory"""
        self.long_term.store_memory(fact, memory_type=memory_type)
        logger.info(f"Stored memory: {fact}")

    def reflect(self) -> str:
        """Reflect on recent actions"""
        if not self.action_history:
            return ""

        recent_actions = self.action_history[-5:]  # Last 5 actions
        history_text = "Recent actions:\n"
        for i, action in enumerate(recent_actions):
            history_text += f"{i+1}. {action['tool']} → {action.get('success', 'unknown')}\n"

        reflection_prompt = f"""{history_text}

Quick reflection: Are we making progress? Any issues?"""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": reflection_prompt}]
        )

        return response.choices[0].message.content

    def execute_task(self, user_message: str, use_long_term: bool = True, max_iterations: int = 10):
        """
        Execute a task with full capabilities
        """
        print(f"\n{'='*70}")
        print(f"{self.name}: EXECUTING TASK")
        print('='*70)
        print(f"Task: {user_message}\n")

        # Start task in working memory
        self.working.start_task(user_message)

        # Retrieve relevant long-term memories
        if use_long_term and self.long_term.collection.count() > 0:
            memories = self.long_term.retrieve_memories(user_message, n_results=3)
            if memories and memories[0]['relevance'] > 0.5:
                print("🧠 Retrieved memories:")
                memory_context = "Relevant memories:\n"
                for mem in memories[:3]:
                    if mem['relevance'] > 0.5:
                        print(f"   • {mem['content']}")
                        memory_context += f"- {mem['content']}\n"

                # Add to short-term memory
                self.short_term.add_user_message(f"{memory_context}\nTask: {user_message}")
            else:
                self.short_term.add_user_message(user_message)
        else:
            self.short_term.add_user_message(user_message)

        # Main execution loop
        iteration = 0

        while iteration < max_iterations:
            iteration += 1
            print(f"\n{'─'*60}")
            print(f"Iteration {iteration}")
            print('─'*60)

            # Reflection every 3 iterations
            if iteration > 1 and iteration % 3 == 0:
                reflection = self.reflect()
                print(f"\n🤔 Reflection: {reflection[:100]}...")
                self.reflection_history.append(reflection)

            # Get response
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=self.short_term.get_messages(),
                    tools=self.tools,
                    tool_choice="auto"
                )

                response_message = response.choices[0].message

                # Check if done
                if not response_message.tool_calls:
                    final_answer = response_message.content
                    self.short_term.add_assistant_message(final_answer)
                    self.working.complete_task(success=True)

                    print(f"\n✅ COMPLETE")
                    print(f"\n{self.name}: {final_answer}")
                    print('='*70)

                    return {
                        "success": True,
                        "answer": final_answer,
                        "iterations": iteration,
                        "actions": len(self.action_history),
                        "reflections": len(self.reflection_history)
                    }

                # Show reasoning
                if response_message.content:
                    print(f"💭 {response_message.content[:100]}...")

                # Execute tools
                self.short_term.messages.append(response_message)

                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    arguments = json.loads(tool_call.function.arguments)

                    print(f"\n🔧 {function_name}({json.dumps(arguments)})")

                    # Execute
                    result = self.functions[function_name](**arguments)
                    success = result.get("success", True) if isinstance(result, dict) else True

                    print(f"   Result: {json.dumps(result)}")

                    # Record action
                    self.action_history.append({
                        "iteration": iteration,
                        "tool": function_name,
                        "args": arguments,
                        "result": result,
                        "success": success
                    })

                    # Store in working memory if result has data
                    if success and isinstance(result, dict) and "result" in result:
                        self.working.set_variable(f"{function_name}_result", result["result"])

                    # Add to conversation
                    self.short_term.messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": json.dumps(result)
                    })

            except Exception as e:
                logger.error(f"Error in iteration {iteration}: {str(e)}")
                self.working.complete_task(success=False)
                return {
                    "success": False,
                    "error": str(e),
                    "iterations": iteration
                }

        # Max iterations reached
        self.working.complete_task(success=False)
        return {
            "success": False,
            "error": "Max iterations reached",
            "iterations": iteration
        }

print("IntelliAgent v1.0 class defined!")

In [ ]:
# Demonstrate IntelliAgent Capabilities

print("\n" + "="*70)
print("🧠 INTELLIAGENT v1.0 - INTELLIGENT AGENT SYSTEM")
print("="*70)

agent = IntelliAgent(name="IntelliAgent")

# Store some long-term memories
print("\n📝 Storing long-term memories...")
agent.remember("User's name is Bob", memory_type="user_fact")
agent.remember("User loves data science", memory_type="user_preference")

# Test tasks
test_tasks = [
    "Calculate 15% of 600",
    "What is Python and when was it created?",
    "Calculate 20% of 500, then search for info about machine learning",
]

for i, task in enumerate(test_tasks, 1):
    print(f"\n{'#'*70}")
    print(f"TASK {i}/{len(test_tasks)}")
    print('#'*70)

    result = agent.execute_task(task)

    print(f"\n📊 Task Summary:")
    print(f"   Success: {result['success']}")
    print(f"   Iterations: {result.get('iterations', 0)}")
    print(f"   Actions: {result.get('actions', 0)}")
    print(f"   Reflections: {result.get('reflections', 0)}")

print("\n" + "="*70)
print("✅ DEMONSTRATION COMPLETE")
print("="*70)
print(f"\nIntelliAgent v1.0 Statistics:")
print(f"  • Total actions: {len(agent.action_history)}")
print(f"  • Reflections: {len(agent.reflection_history)}")
print(f"  • Long-term memories: {agent.long_term.collection.count()}")
print("="*70)

## 🎉 Lab Complete!

Congratulations! You've built a complete intelligent agent system.

### What You Accomplished:

1. **Memory systems** - Short-term, working, and long-term memory
2. **ReAct agents** - Transparent reasoning with Thought → Action → Observation
3. **Planning agents** - Create strategies before executing
4. **Reflective agents** - Self-evaluate and adjust course
5. **IntelliAgent v1.0** - Complete intelligent agent system

### Key Concepts Mastered:

**Memory:**
- Short-term memory for conversation history
- Working memory for task state
- Long-term memory with vector databases
- Memory retrieval with semantic search

**Planning & Reasoning:**
- ReAct framework (Reasoning + Acting)
- Plan-then-execute pattern
- Self-reflection and error correction
- Dynamic replanning

**Production Skills:**
- Error handling in agents
- Logging and monitoring
- Modular agent architecture
- Memory management strategies

## Additional Challenges

Ready to extend IntelliAgent? Try these:

### Challenge 1: Hierarchical Planning
Build an agent that:
1. Breaks complex tasks into subtasks
2. Creates plans for each subtask
3. Executes in dependency order
4. Tracks progress hierarchically

### Challenge 2: Memory Decay
Implement memory decay where:
1. Old memories become less relevant over time
2. Relevance score decreases with age
3. Very old memories are archived or deleted
4. Important memories are preserved

### Challenge 3: Multi-Agent Collaboration
Create multiple specialized agents that:
1. Each have their own memory and expertise
2. Communicate and share information
3. Delegate tasks to appropriate agent
4. Combine results from multiple agents

### Challenge 4: Dynamic Replanning
Add replanning when:
1. Actions fail repeatedly
2. Goals change mid-execution
3. New information becomes available
4. Resource constraints are hit

### Challenge 5: Memory Consolidation
Implement:
1. Merge similar memories to reduce duplication
2. Summarize old conversations before archiving
3. Extract key facts from conversation history
4. Store failed approaches to avoid repeating mistakes

## Key Takeaways

### Memory Architecture:
- Use **short-term memory** for conversation context (trimmed)
- Use **working memory** for current task state
- Use **long-term memory** for persistent facts (vector DB)
- Combine all three for complete agent memory

### Agent Patterns:
- **ReAct**: Explicit reasoning makes agents debuggable
- **Planning**: Upfront planning prevents inefficient paths
- **Reflection**: Periodic self-evaluation enables adaptation
- **Combination**: Real agents use all patterns together

### Production Considerations:
- Always implement error handling and logging
- Set reasonable iteration limits to prevent loops
- Monitor agent behavior and costs
- Use reflection to detect and recover from failures
- Test edge cases and failure scenarios

### Next Steps:
- Explore async agent execution for parallelism
- Implement streaming responses for better UX
- Add conversation memory and context windows
- Build agent analytics and monitoring dashboards
- Deploy to production with proper scaling